In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
import sys
import pickle

sys.modules.pop('generate_syn_data', None)
from generate_syn_data import *

sys.modules.pop('ARW', None)
from ARW import *

sys.modules.pop('split', None)
from split import *

sys.modules.pop('algo_syn', None)
from algo_syn import *

sys.modules.pop('arxiv_read', None)
from arxiv_read import *

In [2]:
# # read data
# reader = readArxiv()
# num_periods = 200
# end = '20231231'
# window_width = 7
# category_list = ['cs.LG']

# abs_list, B_arr = reader.get_abstracts(num_periods = num_periods, end=end, window_width=window_width,category_list=category_list)

# ngrams = [('deep',), ('neural',), ('dnn',), ('dnns',)]
# X = reader.get_X_data(abs_list = abs_list, ngrams = ngrams, task='count')

# #save X and B_arr
# with open('./pickle/X_DL.pkl', 'wb') as f:
#     pickle.dump(X, f)
# with open('./pickle/B_arr_DL.pkl', 'wb') as f:
#     pickle.dump(B_arr, f)

In [ ]:
num_periods = 200
end = '20231231'
window_width = 7
category_list = ['cs.LG']

with open('./pickle/X_DL.pkl', 'rb') as f:
    X = pickle.load(f)
with open('./pickle/B_arr_DL.pkl', 'rb') as f:
    B_arr = pickle.load(f)
    

np.random.seed(20)
B_arr = B_arr.astype(int)

# clip the validation B_arr to have at most (N) per period
B_arr_val = np.floor(B_arr).astype(int)
B_arr_val = np.clip(B_arr_val, 0, 5)
print('maximum # B_val per period:', max(B_arr_val), '\nminimum # B_val per period:', min(B_arr_val), '\nmean # B_val per period:', np.mean(B_arr_val))

B_arr_train = 3 * B_arr_val
B_arr_test = B_arr - B_arr_train - B_arr_val
print('\nmaximum # B_test per period:', max(B_arr_test), '\nminimum # B_test per period:', min(B_arr_test), '\nmean # B_test per period:', np.mean(B_arr_test))

# compute the start and end indices for each period
B_arr_train_starts = np.cumsum(B_arr_train) - B_arr_train
B_arr_train_ends = np.cumsum(B_arr_train) - 1
B_arr_val_starts = np.cumsum(B_arr_val) - B_arr_val
B_arr_val_ends = np.cumsum(B_arr_val) - 1
B_arr_test_starts = np.cumsum(B_arr_test) - B_arr_test
B_arr_test_ends = np.cumsum(B_arr_test) - 1

# sample train, assess and test data according to the B_arr's
X_train, X_val, X_test = sample_X(X=X, B_arr = B_arr, B_arr_train=B_arr_train, B_arr_assess=B_arr_val)
print(X_train.shape, X_val.shape, X_test.shape)

In [4]:
# initialize dictionary storing excess risk
window_sizes_train = [1, 4, 16, 64, 256]
window_sizes_val = [1, 4, 16, 64, 256]

num_seeds = 20
seeds = np.arange(num_seeds) + 2024

excess_dict = {}
excess_dict['ARW'] = {}
for k in window_sizes_val:
    excess_dict[f'Val_{k}'] = {}
    for trial in range(len(seeds)):
        excess_dict[f'Val_{k}'][trial] = []
        excess_dict['ARW'][trial] = []

In [ ]:
delta = 0.1
M = 0
test_means = []

# for each period, train and select the best model
for trial, seed in tqdm(enumerate(seeds)):
    print('Trial:', trial)
    np.random.seed(seed)
    
    # resample train, assess and test data according to the B_arr's
    X_train, X_val, X_test = sample_X(X=X, B_arr = B_arr, B_arr_train=B_arr_train, B_arr_assess=B_arr_val)

    # record the test means
    test_means = []

    for t in range(num_periods):

        # get the t-th period test data
        X_test_t = X_test[B_arr_test_starts[t]:B_arr_test_ends[t]+1]

        # compute the mean of the t-th period test data
        test_mean = np.mean(X_test_t)
        test_means.append(test_mean)

        # get the first t periods of training and validation data
        X_train_t = X_train[:B_arr_train_ends[t]+1]
        B_arr_train_t = B_arr_train[:t+1]
        X_val_t = X_val[:B_arr_val_ends[t]+1]
        B_arr_val_t = B_arr_val[:t+1]

        #train fixed-window models
        models = train_synthetic(X_train_t, B_arr_train_t, window_sizes_train)

        #model selection using fixed-window validation data
        indices_selected, models_selected = select_synthetic_fixed(X_val_t, B_arr_val_t, models, window_sizes_val)

        for (i, k) in enumerate(window_sizes_val):
            mu_hat_fixed_i = models_selected[i]
            excess_risk = (test_mean - mu_hat_fixed_i) ** 2
            excess_dict[f'Val_{k}'][trial].append(excess_risk)
    
        # model selection using rolling window
        seed = 2024
        idx_ARW, mu_hat_ARW = select_synthetic_ARW(X_val_t, B_arr_val_t, models, delta, M, seed)
        excess_risk = (test_mean - mu_hat_ARW) ** 2
        excess_dict['ARW'][trial].append(excess_risk)

    # plot test means
    fig, ax = plt.subplots(1, 1, figsize = (5,5))
    plt.plot(test_means)
    plt.xlabel('Time')
    plt.ylabel('Estimated Mean')
    plt.tight_layout()
    # change figure name accordingly
    plt.savefig(f'./figures/test_means_{trial}.png', dpi=300, bbox_inches='tight')


In [ ]:
start_time = 0   # start time for plotting

methods = list(excess_dict.keys())
excess_array = np.zeros((len(methods), len(seeds), num_periods))

for (i, method) in enumerate(excess_dict.keys()):
    for (j, trial) in enumerate(excess_dict[method].keys()):
        excess_array[i, j, :] = excess_dict[method][trial]

# take average of excess risks over time and trials
mean_excess_over_time_and_trial = np.mean(np.mean(excess_array[:, :, start_time:], axis=2), axis=1)
mean_std_mse_over_time_and_trial = np.mean(np.std(excess_array[:, :, start_time:], axis=2), axis=1)

# bar plot
fig, ax = plt.subplots(1, 1, figsize = (5, 5))
methods = list(excess_dict.keys())
colors = ['tab:red', '#FFA500', 'tab:purple', 'tab:brown', 'tab:green', '#0096FF']
for pos, method in enumerate(methods):
    # bar plot the mean excess risk
    plt.bar(pos, mean_excess_over_time_and_trial[pos], color=colors[pos])
    print(method, mean_excess_over_time_and_trial[pos])
mpl.rcParams.update({'font.size': 12})
plt.xticks(np.arange(6), ['ARW', 'V1', 'V4', 'V16', 'V64', 'V256'])
plt.xlabel('Model Selection Algorithms')
plt.ylabel('Mean Excess Risk')
plt.tight_layout()
plt.savefig(f'./figures/bar.png', dpi=300, bbox_inches='tight')


# line plot with 3 algorithms
fig, ax = plt.subplots(1, 1, figsize = (6, 2.5))
plt.plot(excess_dict['ARW'][0][start_time:], color='r')
plt.plot(excess_dict['Val_1'][0][start_time:], color='#FFA500')
plt.plot(excess_dict['Val_256'][0][start_time:], color='#0096FF')
mpl.rcParams.update({'font.size': 12})
plt.xlabel('Time')
plt.ylabel('Excess Risk')
plt.tight_layout()
plt.savefig(f'./figures/line_few.png', dpi=300, bbox_inches='tight')

# line plot with many algorithms
fig, ax = plt.subplots(1, 1, figsize = (6, 2.5))
plt.plot(excess_dict['ARW'][0][start_time:], color='r')
plt.plot(excess_dict['Val_1'][0][start_time:], color='#FFA500')
plt.plot(excess_dict['Val_4'][0][start_time:], color='purple')
plt.plot(excess_dict['Val_16'][0][start_time:], color='brown')
plt.plot(excess_dict['Val_64'][0][start_time:], color='green')
plt.plot(excess_dict['Val_256'][0][start_time:], color='#0096FF')
mpl.rcParams.update({'font.size': 12})
plt.xlabel('Time')
plt.ylabel('Excess Risk')
plt.tight_layout()
plt.savefig(f'./figures/line_many.png', dpi=300, bbox_inches='tight')

In [ ]:
mean_excess_over_time_and_trial